In [173]:
import requests
import csv
import re
import random
import pandas as pd
import time
from bs4 import BeautifulSoup
import os
import numpy as np
from numpy import arange
import string

In [174]:
#list of user agents to cycle through
user_agent_list = [
   #Chrome
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

In [49]:
#headers for scraper
headers = {
    'user-agent': random.choice(user_agent_list)
}

In [2]:
inexp_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [3]:
mod_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [4]:
pricey_page1 = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.3&ed_attrs=RestaurantsPriceRange2.1'

***Pages of reviews for each resturant*** <br>
1) We can use all characters between "datePublished.....and... /iv>" to capture the actual texts of reviews <br>
2) We can also use "ratingValue" content=" as a marker for star rating <br>
3) we will have to use the total number of reviews (found in plain html on the first page of all resturants) to calculate how many pages of reviews (assuming 20 reviews per page) 

***Pages of resturants for each price point***<br>
1) We can manually scrape total pages and total number of resturants and then we assume 30 resturants per page <br>
2) We will have to also scrape all resturant URLs for individual resturants for pre-review scraping <br>
3) We can use "ranking"....."name"......"neighbohoods" as markers for each resturant name we have to scrape and process

***Scrapper Pseudo Code***<br>
1- Calculate the number of pages of resturants each price point has <br>
2- Extract URLS and process all resturant names for each price point <br>
3- Calculate the number of pages of reviews each resturant has <br>
4- Use numbers from above to establish dataframes and loops <br>
5- Finally scrape all reviews and star ratings into 4 seperate dataframes, one for each price point </br>

In [5]:
#Put price point URL pages in list
price_points = list()
price_points.append(inexp_page1)
price_points.append(mod_page1)
price_points.append(pricey_page1)

In [43]:
price_points[0]

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.1'

In [6]:
#list for resturants in each price point
#pp_rest = list()

In [19]:
#get number of pages of resturants for each price point
#proxies1 = {
#  'https': '114.7.206.225:8080',
#}
#for price_point in price_points:
#    response = requests.get(price_point, proxies =proxies1, timeout=5,headers=headers)
#    content = response.content
#    pp_rest.append(re.findall('Showing 1-30 of....',str(content)))

In [20]:
#content

***These are manually copy and pasted from the website***

In [27]:
price_rests = [446,820,107]
#pp_rest

In [22]:
#dictionary for pages
#price_rests = dict()
#extract last 3 numbers
#pp_rest[0] = str(pp_rest[0])
#rests = re.findall('[0-9]{3}',str(pp_rest[0]))
#rests = str(rests)
#rests = rests[2:5]
#price_rests['inexp'] = int(rests)
#---------------------------------------
#pp_rest[1] = str(pp_rest[1])
#rests = re.findall('[0-9]{3}',str(pp_rest[1]))
#rests = str(rests)
#rests = rests[2:5]
#price_rests['mod'] = int(rests)
#---------------------------------------
#pp_rest[2] = str(pp_rest[2])
#rests = re.findall('[0-9]{3}',str(pp_rest[2]))
#rests = str(rests)
#rests = rests[2:5]
#price_rests['pricey'] = int(rests)

In [37]:
price_rests

[446, 820, 107]

In [40]:
#calculate number of pages using number of resturants
pp_pages = dict()
ppoint = 1
for pp in price_rests:
    pages = (pp/30) + 1
    pp_pages[ppoint] = int(pages)
    ppoint+=1

In [175]:
pp_pages

{1: 15, 2: 28, 3: 4}

Price point 1: 15 pages, price point 2: 28 pages, price point 3: 4 pages

## 2.0 Extract and Process all resturant URLs for each price point
### 2.1 Cheapest price point (Inexp)
#### first we construct all URLs for each page of resturants
After examining the URLs,***'&start=30'*** is added to the end of the url for page 2 (30 is added for every page)
The last page url of any price point is therefore constructed by adding ***'&start=(MaxPages-1)*30'***, so
***page2 = inexp_page1 + '&start=30'***

In [192]:
#lets now save this as resturant count, page 2 = 30, page 3 = 60....page 15 = 420
resturant_count = list()
#construct count to add to URL
for x in range(1,15):
    resturant_count.append(str((x*30)))

In [ ]:
dirty_links = list()
dirtynames = re.findall('href="/biz/(.*?)" t',page1)
dirty_links.append(dirtynames)
links_set = set(dirty_links[0])
counter = 0
for link in links_set:
    if 'hrid' in link:
        counter+=1
        print('https://www.yelp.com/biz/'+link)

In [220]:
#Get resturant names from each page, start with page 1 then loop through the rest
dirty_links = list()
proxies = {
  'https': '104.129.202.118:8800',
}
response = requests.get(inexp_page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=10)
content = response.content
copycontent = str(content)
dirtylinks = list()
dirtylinks = re.findall('href="/biz/(.*?)" t',copycontent)
dirty_links.append(dirtylinks)
for count in resturant_count:
    time.sleep(10)
    new_page = inexp_page1 + '&start=' + count
    response = requests.get(new_page,headers={'user-agent': random.choice(user_agent_list)}, timeout=10)
    content = response.content
    copycontent = str(content)
    dirtylinks = list()
    dirtylinks = re.findall('href="/biz/(.*?)" t',copycontent)
    dirty_links.append(dirtylinks)
    print('Page : ',int(int(count)/30-1))#Print this to know how far along the scraping is

Page :  0
Page :  1
Page :  2
Page :  3
Page :  4
Page :  5
Page :  6
Page :  7
Page :  8
Page :  9
Page :  10
Page :  11
Page :  12
Page :  13


In [221]:
inexp_clean_links = list()
for linkset in dirty_links:
    links_set = set(linkset)
    for link in links_set:
        if 'hrid' in link:
            link = 'https://www.yelp.com/biz/'+link
            inexp_clean_links.append(link)

In [222]:
len(inexp_clean_links)

391

Some links need an extra round of cleaning:

In [231]:
num = 0
linkindex = list()
for link in inexp_clean_links:
    num = num + 1
    if len(link) > 500:
        linkindex.append(num)

In [240]:
for index in linkindex:
    dirtylinks = list()
    dirtylinks = re.findall('href="/biz/(.*?)" ',str(inexp_clean_links[index]))
    inexp_clean_links[index] = dirtylinks
    print(dirtylinks)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [239]:
inexp_clean_links

['https://www.yelp.com/biz/plaza-noodle-strathfield?hrid=yzYqR2PxwdBdrZdEUa55bg&amp;osq=Restaurants',
 'https://www.yelp.com/biz/knafeh-parramatta?hrid=d8U8bufzE0BDzoTg_g-9LQ&amp;osq=Restaurants',
 'https://www.yelp.com/biz/tasty-house-chinese-restaurant-ashfield?hrid=eSJ7JrTvvWLbwxvd5nQdOQ&amp;osq=Restaurants',
 'https://www.yelp.com/biz/banh-cuon-kim-thanh-bankstown?hrid=GZ3c5t_tKIcrUmyIfo_Ahw&amp;osq=Restaurants',
 'https://www.yelp.com/biz/winston-hills-mall-winston-hills-3">Winston Hills Mall</a></div></div><div class="lemon--div__373c0__1mboc u-space-b1 border-color--default__373c0__2oFDT"></div></div></div></div><div class="lemon--div__373c0__1mboc u-space-t2 u-space-b2 border-color--default__373c0__2oFDT"><div class="lemon--div__373c0__1mboc arrange__373c0__UHqhV vertical-align-middle__373c0__2TQsQ border-color--default__373c0__2oFDT"><div class="lemon--div__373c0__1mboc arrange-unit__373c0__1piwO arrange-unit-fill__373c0__17z0h border-color--default__373c0__2oFDT"><p class="le

In [67]:
#Get resturant names from each page, start with page 1 then loop through the rest
proxies2 = {
  'https': '82.200.65.90:8080',
}
response = requests.get(inexp_page1,proxies=proxies2)
content = response.content
copycontent = str(content)
dirtynames = re.findall('target="" name=".{75}',copycontent)
dirty_names.append(dirtynames)
for count in resturant_count:
    time.sleep(10)
    new_page = inexp_page1 + '&start=' + count
    response = requests.get(new_page,headers={'user-agent': random.choice(user_agent_list)}, timeout=10)
    content = response.content
    copycontent = str(content)
    dirtynames = list()
    dirtynames = re.findall('target="" name=".{75}',copycontent)
    dirty_names.append(dirtynames)

In [ ]:
dirty_names[0][:10]

In [70]:
#the captured strings need to be cleaned once more to extract only the resturant names:
clean_names = list()
for name in dirty_names:
    for string in name:
        cleanstring = re.findall('name="(.*?)"',string)
        clean_names.append(cleanstring)

In [71]:
clean_names[:10]

[[''],
 [''],
 [''],
 [''],
 ['El Jannah Granville'],
 [''],
 [''],
 [''],
 ['Frango'],
 ['']]

In [72]:
#we then need to convert the resturant names to strings
final_names = list()
for name in clean_names:
    name = str(name)
    name = name[2:]
    final_names.append(str(name))
    #print(name)

In [73]:
#now we need to clean the final names
clean_finalnames = list()
for name in final_names:
    if name[0] != "'":
        clean_finalnames.append(name[:len(name)-2])
        print(name)

El Jannah Granville']
Frango']
Aldhiaffah Al-Iraqi Restaurant']
Tan Viet Noodle House']
The Original Frango']
Industrial Harvest']
Darband Restaurant']
Hawa Charcoal Chicken']
Pho Tau Bay']
Bau Truong Restaurant']
The Usual Cafe']
Al Shami - Syrian &amp; Lebanese Cuisine']
Pho Ba Le Vietnamese Noodle Bar']
Anema E Core Pizzeria']
Mr Ping\\xe2\\x80\\x99s']
Pho Mai']
Pho Sam Noodle House']
Gourmet Noodle House']
Awafi Charcoal Chicken']
Jasmin Lebanese Restaurant']
El Burger']
El Jannah']
New Shanghai']
Lao Village']
Rashays']
Knafeh']
Banh Cuon Kim Thanh']
S Biber &amp; T Biber']
Plaza Noodle']
Bar Qualia']
New Star Kebabs']
Chubby Buns Burger']
Pho Gia Hoi']
Mr Happy Chef Noodle House']
Simply Noodles']
The Shed by Ristretto &amp; Co']
Pho Saigon']
Tasty House Chinese Restaurant']
Darcy Street Project']
Pho Lam Vietnamese Restaurant']
Pho Ann']
Jasmin 1 lebanese restaurant chesterhill']
Oriental Tucker Box']
Lucky Thai &amp; Lao Restaurant']
Hana Sushi']
Traditional Gyradiko']
Badmanne

In [74]:
len(clean_finalnames)

446

446 is the correct number of resturants, we just need to do 3 more things. Remove the - '] from the end of every name and replace \\xe2\\x80\\x99 with ' and replace amp& with nothing

In [75]:
#final phase of cleaning
inexp_names = list()
for name in clean_finalnames:
    _name = re.sub("]'",'',name)
    clean_name = re.sub("amp;",'',_name)
    final_name = re.sub(r'\\\\xe2\\\\x80\\\\x99',"'",clean_name)
    inexp_names.append(final_name)

In [77]:
counter = 1
for name in inexp_names:
    print(name)
    print(counter)
    counter = counter + 1

El Jannah Granville
1
Frango
2
Aldhiaffah Al-Iraqi Restaurant
3
Tan Viet Noodle House
4
The Original Frango
5
Industrial Harvest
6
Darband Restaurant
7
Hawa Charcoal Chicken
8
Pho Tau Bay
9
Bau Truong Restaurant
10
The Usual Cafe
11
Al Shami - Syrian & Lebanese Cuisine
12
Pho Ba Le Vietnamese Noodle Bar
13
Anema E Core Pizzeria
14
Mr Ping's
15
Pho Mai
16
Pho Sam Noodle House
17
Gourmet Noodle House
18
Awafi Charcoal Chicken
19
Jasmin Lebanese Restaurant
20
El Burger
21
El Jannah
22
New Shanghai
23
Lao Village
24
Rashays
25
Knafeh
26
Banh Cuon Kim Thanh
27
S Biber & T Biber
28
Plaza Noodle
29
Bar Qualia
30
New Star Kebabs
31
Chubby Buns Burger
32
Pho Gia Hoi
33
Mr Happy Chef Noodle House
34
Simply Noodles
35
The Shed by Ristretto & Co
36
Pho Saigon
37
Tasty House Chinese Restaurant
38
Darcy Street Project
39
Pho Lam Vietnamese Restaurant
40
Pho Ann
41
Jasmin 1 lebanese restaurant chesterhill
42
Oriental Tucker Box
43
Lucky Thai & Lao Restaurant
44
Hana Sushi
45
Traditional Gyradiko
46
B

The Fusion Resturant name is currently saved as: ***"Fusion \\xe7\\x85\\xae\\xe6\\x84\\x8f"***, this is because there are two chinese characters following the Fusion name, it is represented in the browser as : ***"fusion %E7%85%AE%E6%84%8F"***, so we will replace this.

In [80]:
inexp_names[129] = 'Fusion %E7%85%AE%E6%84%8F'

In [81]:
inexp_names[129]

'Fusion %E7%85%AE%E6%84%8F'

The comma in ***Coffee in the park, Dolls Point*** must also be removed since commas are our delimiter for the text file that we will write our resturant names to.

In [87]:
inexp_names[435] = 'Coffee in the park Cafe Dolls Point'
inexp_names[435]

'Coffee in the park Cafe Dolls Point'

In [88]:
#Finally, write all names to csv file
np.savetxt("inexpensive_names.csv", inexp_names, delimiter=",", fmt='%s', header='Resturant Names')

### 2.2 Moderate Price Point
There are 28 pages of resturants in this price point

In [89]:
#save resturant count, page 2 = 30, page 3 = 60....page 28 = 810
resturant_count = list()
#declare another list to keep all scrapped dirty names from the inexp price point
dirty_names = list()

In [90]:
for x in range(1,28):
    resturant_count.append(str((x*30)))

In [92]:
#Get resturant names from each page, start with page 1 then loop through the rest
proxies = {
  'https': '143.255.52.90:8080',
}
response = requests.get(mod_page1,proxies=proxies)
content = response.content
copycontent = str(content)
dirtynames = re.findall('target="" name=".{75}',copycontent)
dirty_names.append(dirtynames)
for count in resturant_count:
    time.sleep(10)
    new_page = mod_page1 + '&start=' + count
    response = requests.get(new_page,headers={'user-agent': random.choice(user_agent_list)}, timeout=10)
    content = response.content
    copycontent = str(content)
    dirtynames = list()
    dirtynames = re.findall('target="" name=".{75}',copycontent)
    dirty_names.append(dirtynames)
    print(count,"out of",resturant_count)

In [95]:
len(dirty_names)

28

In [96]:
#the captured strings need to be cleaned once more to extract only the resturant names:
clean_names = list()
for name in dirty_names:
    for string in name:
        cleanstring = re.findall('name="(.*?)"',string)
        clean_names.append(cleanstring)

In [97]:
clean_names[:10]

[[''],
 [''],
 [''],
 [''],
 ['Me Oi'],
 [''],
 [''],
 [''],
 ['Arthur Street Cafe'],
 ['']]

In [98]:
#we then need to convert the resturant names to strings
final_names = list()
for name in clean_names:
    name = str(name)
    name = name[2:]
    final_names.append(str(name))
    #print(name)

In [99]:
#now we need to clean the final names
clean_finalnames = list()
for name in final_names:
    if name[0] != "'":
        clean_finalnames.append(name[:len(name)-2])
        print(name)

Me Oi']
Arthur Street Cafe']
Kanzo Fresh Sushi']
Chatkaaz']
The Baron']
El Jannah Blacktown']
Smokkim']
Belmonte\\xe2\\x80\\x99s Pizzeria Restaurante']
Hooters Of Parramatta']
Abyssina Ethiopian Restaurant']
Nieo\\xe2\\x80\\x99s Grille']
The Local Shed Cafe']
Holy Basil']
The Middle Feast']
VN Fresh']
Temasek Restaurant']
Coco Cubano']
Pak Hailam Kopitiam']
Big Tree House Cafe']
The Butcher\\xe2\\x80\\x99s Block']
Outback Steakhouse']
Memphis Wings']
Royal Cricketers Arms']
The Elevation']
The Winston']
257 Home Kitchen']
Ray &amp; Lou\\xe2\\x80\\x99s Steak &amp; Seafood Restaurant']
Dos Senoritas']
Waterview - Dooleys at Silverwater']
Ray &amp; Lou\\xe2\\x80\\x99s Steak &amp; Seafood Restaurant']
Grill\\xe2\\x80\\x99d']
Jaam E Jam']
VN Fresh']
Taste of Texas BBQ']
Fortune8 Chinese Restaurant']
Basement Brewhouse']
Waterview - Dooleys at Silverwater']
Mr Stonebowl']
Restaurant 3one7']
Volcano\\xe2\\x80\\x99s steakhouse']
Star Buffet']
Sahara']
German Butchery']
The Cup &amp; Plate Cafe

The Picnic Burwood']
Mitzu']
Tokyo Japanese Restaurant']
KFC']
Thornleigh Curry House']
La Vigne Bakery']
At Thai Tonight']
Iconic Cafe']
Titanic Cafe']
Taste of Malaya']
Khoune Kham']
Indian Home Kitchen']
Cafe Centro']
Napoli Centro Pizzeria']
S.A.L.T Cafe']
Friendship Oriental Seafood Restaurant']
Loft Cafe']
Lan Yuan']
Bar Cortona']
Coffee Tank']
Gemelle Ristorante Italiano']
Janani']
Dolcissimo']
Blends &amp; Brothers']
Sydney Dumpling King']
Com Tam Ngon']
Curry Sutra']
Mad Mex']
Karoo &amp; Co']
Ichiraku Japanese Noodle Bar']
Mint Fresh IGA']
Ceci']
Raj Mahal Indian Restaurant']
Saigon Spice']
Taj Indian Sweets &amp; Restaurant']
Peking Restaurant']
D\\xe2\\x80\\x99roost']
Authentic Thai Delight']
Zaitoune Mezza and Grill']
Beauty and the Beast Cafe &amp; Restaurant']
Biviano\\xe2\\x80\\x99s Italian Restaurant']
UTS Haberfield Club']
Flat Rock Cafe']
Myssy\\xe2\\x80\\x99s Fresh &amp; Cooked Seafood']
The Hub House Diner']
King\\xe2\\x80\\x99s Seafood Restaurant']
Wangfu Family K

In [100]:
len(clean_finalnames)

789

In [101]:
#final large phase of cleaning
mod_names = list()
for name in clean_finalnames:
    _name = re.sub("]'",'',name)
    clean_name = re.sub("amp;",'',_name)
    final_name = re.sub(r'\\\\xe2\\\\x80\\\\x99',"'",clean_name)
    mod_names.append(final_name)

In [103]:
counter = 1
for name in mod_names:
    print(name,':',counter)
    counter = counter + 1

Me Oi : 1
Arthur Street Cafe : 2
Kanzo Fresh Sushi : 3
Chatkaaz : 4
The Baron : 5
El Jannah Blacktown : 6
Smokkim : 7
Belmonte's Pizzeria Restaurante : 8
Hooters Of Parramatta : 9
Abyssina Ethiopian Restaurant : 10
Nieo's Grille : 11
The Local Shed Cafe : 12
Holy Basil : 13
The Middle Feast : 14
VN Fresh : 15
Temasek Restaurant : 16
Coco Cubano : 17
Pak Hailam Kopitiam : 18
Big Tree House Cafe : 19
The Butcher's Block : 20
Outback Steakhouse : 21
Memphis Wings : 22
Royal Cricketers Arms : 23
The Elevation : 24
The Winston : 25
257 Home Kitchen : 26
Ray & Lou's Steak & Seafood Restaurant : 27
Dos Senoritas : 28
Waterview - Dooleys at Silverwater : 29
Ray & Lou's Steak & Seafood Restaurant : 30
Grill'd : 31
Jaam E Jam : 32
VN Fresh : 33
Taste of Texas BBQ : 34
Fortune8 Chinese Restaurant : 35
Basement Brewhouse : 36
Waterview - Dooleys at Silverwater : 37
Mr Stonebowl : 38
Restaurant 3one7 : 39
Volcano's steakhouse : 40
Star Buffet : 41
Sahara : 42
German Butchery : 43
The Cup & Plate Ca

Vogue Cafe : 365
The Vintage Goose : 366
Inflames Charcoal Grill : 367
Pron Prohm Thai Restaurant : 368
Saravanaa Bhavan : 369
AJ's Indian Restaurant : 370
Choco Cannelle : 371
Canley Heights Bistro : 372
The Hills Club : 373
Huong Xua : 374
Frankies Food Factory : 375
Beijing Station Noodle : 376
Thai Kitchen : 377
Cafe Bones : 378
The Picnic Burwood : 379
Mitzu : 380
Tokyo Japanese Restaurant : 381
KFC : 382
Thornleigh Curry House : 383
La Vigne Bakery : 384
At Thai Tonight : 385
Iconic Cafe : 386
Titanic Cafe : 387
Taste of Malaya : 388
Khoune Kham : 389
Indian Home Kitchen : 390
Cafe Centro : 391
Napoli Centro Pizzeria : 392
S.A.L.T Cafe : 393
Friendship Oriental Seafood Restaurant : 394
Loft Cafe : 395
Lan Yuan : 396
Bar Cortona : 397
Coffee Tank : 398
Gemelle Ristorante Italiano : 399
Janani : 400
Dolcissimo : 401
Blends & Brothers : 402
Sydney Dumpling King : 403
Com Tam Ngon : 404
Curry Sutra : 405
Mad Mex : 406
Karoo & Co : 407
Ichiraku Japanese Noodle Bar : 408
Mint Fresh IGA

La Roue Resturant name is currently saved as: ***"La Roue Caf\\xc3\\xa9"***, this is because there is an accent over the e, it is represented in the browser as : ***"La Roue Caf%C3%A9"***, so we will replace this.

In [106]:
mod_names[483] = 'La Roue Caf%C3%A9'
mod_names[483]

'La Roue Caf%C3%A9'

In [107]:
mod_names

['Me Oi',
 'Arthur Street Cafe',
 'Kanzo Fresh Sushi',
 'Chatkaaz',
 'The Baron',
 'El Jannah Blacktown',
 'Smokkim',
 "Belmonte's Pizzeria Restaurante",
 'Hooters Of Parramatta',
 'Abyssina Ethiopian Restaurant',
 "Nieo's Grille",
 'The Local Shed Cafe',
 'Holy Basil',
 'The Middle Feast',
 'VN Fresh',
 'Temasek Restaurant',
 'Coco Cubano',
 'Pak Hailam Kopitiam',
 'Big Tree House Cafe',
 "The Butcher's Block",
 'Outback Steakhouse',
 'Memphis Wings',
 'Royal Cricketers Arms',
 'The Elevation',
 'The Winston',
 '257 Home Kitchen',
 "Ray & Lou's Steak & Seafood Restaurant",
 'Dos Senoritas',
 'Waterview - Dooleys at Silverwater',
 "Ray & Lou's Steak & Seafood Restaurant",
 "Grill'd",
 'Jaam E Jam',
 'VN Fresh',
 'Taste of Texas BBQ',
 'Fortune8 Chinese Restaurant',
 'Basement Brewhouse',
 'Waterview - Dooleys at Silverwater',
 'Mr Stonebowl',
 'Restaurant 3one7',
 "Volcano's steakhouse",
 'Star Buffet',
 'Sahara',
 'German Butchery',
 'The Cup & Plate Cafe & Restaurant',
 'District 1 H

In [108]:
#finally we copy names to CSV
np.savetxt("moderate_names.csv", inexp_names, delimiter=",", fmt='%s', header='Resturant Names')

In [109]:
mod_page1

'https://www.yelp.com/search?find_desc=Restaurants&find_loc=western%20sydney&l=g%3A151.15106940214844%2C-33.67728783357405%2C150.7418286794922%2C-34.01944832216618&attrs=RestaurantsPriceRange2.2&ed_attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2'

In [111]:
len(mod_names)

789

In [127]:
proxies = {
  'https': '45.195.5.9:8080',
}

In [128]:
response = requests.get(inexp_page1,headers={'user-agent': random.choice(user_agent_list)}, timeout=10, proxies=proxies)
content = response.content
copycontent = str(content)
dirtynames = list()
#dirtynames = re.findall('target="" name=".{75}',copycontent)
#dirty_names.append(dirtynames)
copycontent

'b\'<!doctype html>\\n    <!--[if lt IE 7 ]> <html class="ie6 ie ltie9 ltie8 no-js" lang="en_US"> <![endif]-->\\n    <!--[if IE 7 ]>    <html class="ie7 ie ltie9 ltie8 no-js" lang="en_US"> <![endif]-->\\n    <!--[if IE 8 ]>    <html class="ie8 ie ltie9 no-js" lang="en_US"> <![endif]-->\\n    <!--[if IE 9 ]>    <html class="ie9 ie no-js" lang="en_US"> <![endif]-->\\n    <!--[if (gt IE 9)|!(IE)]><!--> <html class="no-js" lang="en_US"> <!--<![endif]-->\\n        <head>\\n        <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\\n        <link rel="stylesheet" href="https://s3-media1.fl.yelpcdn.com/assets/srv0/error_pages/8a98d2743e15/assets/scss/maintenance-pkg.css">\\n        <title>Yelp</title>\\n            <link rel="mask-icon" sizes="any" href="https://s3-media1.fl.yelpcdn.com/assets/srv0/yelp_styleguide/4374c8fd03d1/assets/img/logos/yelp_burst.svg" content="#c41200">\\n    <link rel="shortcut icon" href="https://s3-media2.fl.yelpcdn.com/assets/srv0/yelp_stylegui

In [129]:
with open('page1.txt', 'r') as file:
    page1 = file.read()

In [130]:
page1

'<!DOCTYPE HTML>\n\n<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]-->\n<!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]-->\n<!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]-->\n<!--[if (gt IE 9)|!(IE)]><!--> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="no-js" lang="en"> <!--<![endif]-->\n    <head>\n        <script>\n            (function() {\n                var main = null;\n\n                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name==="CSRFallbackError");if(

In [154]:
dirty_links = list()
dirtynames = re.findall('href="/biz/(.*?)" t',page1)
dirty_links.append(dirtynames)

In [158]:
links_set = set(dirty_links[0])

In [172]:
counter = 0
for link in links_set:
    if 'hrid' in link:
        counter+=1
        print('https://www.yelp.com/biz/'+link)

https://www.yelp.com/biz/jasmine-1-auburn?hrid=0U0KOCg66RYDfmNrutNPGw&amp;osq=Restaurants
https://www.yelp.com/biz/circa-espresso-parramatta-2?hrid=gF0sF8Iul_VBQ9B9aKdQeQ&amp;osq=Restaurants
https://www.yelp.com/biz/grano-wetherill-park?hrid=NK_bhg7BKYJes1WxAOfAdA&amp;osq=Restaurants
https://www.yelp.com/biz/left-of-field-rhodes?hrid=xz0CfU7Ebe2CRoa4iEGziA&amp;osq=Restaurants
https://www.yelp.com/biz/tk-something-bbq-restaurant-canley-vale-2?hrid=d5PYcLCt_0kgZ-71G4l-Ig&amp;osq=Restaurants
https://www.yelp.com/biz/xs-espresso-wetherill-park?hrid=pDhsqmF1pTJZ_qYigRLEOQ&amp;osq=Restaurants
https://www.yelp.com/biz/the-grand-siam-thai-restaurant-bella-vista?hrid=kPQR1C_DAZZPXbS2Ds-psw&amp;osq=Restaurants
https://www.yelp.com/biz/sahara-turkish-restaurant-parramatta?hrid=UqqQml1DazHNjyamIjoXLA&amp;osq=Restaurants
https://www.yelp.com/biz/ristretto-and-co-liverpool-2?hrid=xaw9BXjvr59Z-3fLZbIjkg&amp;osq=Restaurants
https://www.yelp.com/biz/jeans-chilli-chicken-eastwood?hrid=GDlfUXjlwsB5nbJb2T